# Példa Minta Szálloda- és Repülőjegy Foglaló Ügynök

Ez a megoldás segít repülőjegyeket és szállodát foglalni. A példa egy utazás London Heathrow (LHR) repülőtérről New York JFK repülőtérre 2024. február 20-án, visszaút 2025. február 27-én, gazdaságos osztályon, kizárólag a British Airways járataival. Szeretnék megszállni egy Hilton szállodában New Yorkban, kérem, adja meg a repülőjegy és a szálloda költségeit.


# Az Azure AI Agent Service inicializálása és konfigurációs információk lekérése a **.env** fájlból

### **.env**

Hozz létre egy .env fájlt

A **.env** tartalmazza az Azure AI Agent Service kapcsolatláncát, az AOAI által használt modellt, valamint a megfelelő Google API Search szolgáltatás API-ját, ENDPOINT-ját stb.

- **AZURE_AI_AGENT_MODEL_DEPLOYMENT_NAME** = "Az Azure AI Agent Service Model Deployment neve"

[**NOTE**] Szükséged lesz egy modellre, amelynek 100,000-es Rate Limitje van (Tokenek percenként) és 600-as Rate Limitje (Kérések percenként).

  Modellt az Azure AI Foundry - Model and Endpoint részben szerezhetsz be.

- **AZURE_AI_AGENT_PROJECT_CONNECTION_STRING** = "Az Azure AI Agent Service Projekt Kapcsolatlánca"

  A projekt kapcsolatláncát az AI ​​Foundry Portal képernyőjén, a projekt áttekintésében találhatod meg.

- **SERPAPI_SEARCH_API_KEY** = "A SERPAPI Search API KULCSOD"
- **SERPAPI_SEARCH_ENDPOINT** = "A SERPAPI Search Endpoint-ed"

Az Azure AI Agent Service Model Deployment Name és Project Connection String megszerzéséhez létre kell hoznod az Azure AI Agent Service-t. Ajánlott [ezt a sablont](https://portal.azure.com/#create/Microsoft.Template/uri/https%3A%2F%2Fraw.githubusercontent.com%2Ffosteramanda%2Fazure-agent-quickstart-templates%2Frefs%2Fheads%2Fmaster%2Fquickstarts%2Fmicrosoft.azure-ai-agent-service%2Fstandard-agent%2Fazuredeploy.json) használni a közvetlen létrehozáshoz (***Megjegyzés:*** Az Azure AI Agent Service jelenleg korlátozott régióban érhető el. Ajánlott, hogy [ezt a linket](https://learn.microsoft.com/en-us/azure/ai-services/agents/concepts/model-region-support) tekintsd meg a régió beállításához).

Az Agent-nek hozzáférésre van szüksége a SERPAPI-hoz. Ajánlott [ezen a linken](https://serpapi.com/searches) regisztrálni. A regisztráció után egyedi API KULCSOT és ENDPOINT-ot szerezhetsz be.


# Beállítás

Ahhoz, hogy futtathasd ezt a notebookot, győződj meg róla, hogy telepítetted a szükséges könyvtárakat a `pip install -r requirements.txt` parancs futtatásával.


In [ ]:
from semantic_kernel import __version__

__version__

A Szemantikai Kernel verziójának legalább 1.27.2-nek kell lennie.


Töltse be a .env fájl beállításait és erőforrásait, kérjük, győződjön meg róla, hogy hozzáadta a kulcsokat és beállításokat, valamint létrehozott egy helyi .env fájlt.


In [ ]:
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Bejelentkezés az Azure-ba

Most be kell jelentkezned az Azure-ba. Nyiss meg egy terminált, és futtasd az alábbi parancsot:

```bash
az login
```

Ez a parancs felszólít, hogy add meg az Azure-hitelesítő adataidat, lehetővé téve az Azure AI Agent szolgáltatás megfelelő működését.


# Magyarázat:
Ez egy változó, amely az API-kulcsot tárolja egy SERP (Keresőmotor találati oldal) API szolgáltatás eléréséhez. Az API-kulcs egy egyedi azonosító, amelyet a fiókodhoz kapcsolódó kérések hitelesítésére használnak.

Cél: Ennek a sornak az a célja, hogy az API-kulcsot egy változóban tárolja, így az felhasználható a SERP API szolgáltatás kéréseinek hitelesítésére. Az API-kulcs szükséges a szolgáltatás eléréséhez és keresések végrehajtásához.
Hogyan szerezhetsz SERP API-kulcsot: Az API-kulcs megszerzéséhez kövesd az alábbi általános lépéseket a https://serpapi.com oldalon (a pontos lépések eltérhetnek az általad használt konkrét SERP API szolgáltatástól):

Válassz egy SERP API szolgáltatást: Számos SERP API szolgáltatás érhető el, például SerpAPI, Google Custom Search JSON API és mások. Válaszd ki azt, amelyik leginkább megfelel az igényeidnek.

Regisztrálj egy fiókot: Látogass el a kiválasztott SERP API szolgáltatás weboldalára, és regisztrálj egy fiókot. Lehet, hogy meg kell adnod néhány alapvető információt, és igazolnod kell az e-mail címedet.

Hozz létre egy API-kulcsot: A regisztráció után jelentkezz be a fiókodba, és navigálj az API szekcióhoz vagy irányítópulthoz. Keresd meg az új API-kulcs létrehozására vagy generálására vonatkozó opciót.
Másold az API-kulcsot a .env fájlodba.


In [ ]:
SERP_API_KEY='SERPAPI_SEARCH_API_KEY'

# Magyarázat:
BASE_URL: Ez egy változó, amely a SERP API végpont alap URL-jét tárolja. A BASE_URL változónév egy konvenció, amely jelzi, hogy ez az URL az API-kérések kiindulópontja.
'https://serpapi.com/search':

Ez az URL-karakterlánc, amely a BASE_URL változóhoz van rendelve. Ez képviseli a végpontot, amelyen keresztül keresési lekérdezéseket lehet végrehajtani a SERP API használatával.

# Cél:
Ennek a sornak az a célja, hogy meghatározzon egy konstans értéket, amely az alap URL-t tartalmazza a SERP API-hoz. Ez az URL lesz a kiindulópont az API-kérések felépítéséhez, amelyek keresési műveleteket hajtanak végre.

# Használat:
Az alap URL-t egy változóban meghatározva könnyen újra felhasználhatod a kódodban, amikor csak szükséged van a SERP API-hoz való kérések végrehajtására. Ezáltal a kód karbantarthatóbbá válik, és csökken a hibák kockázata, amelyek az URL többszöri, kézi beírásából adódhatnak. A jelenlegi példa a https://serpapi.com/search?engine=bing, amely a Bing keresési API-t használja. Különböző API-k választhatók a https://Serpapi.com oldalon.


In [ ]:
BASE_URL = 'https://serpapi.com/search?engine=bing'

# Magyarázat:

Itt található a plugin kódja.

Osztálydefiníció: `class BookingPlugin`: Egy BookingPlugin nevű osztályt definiál, amely tartalmazza a szállodák és repülőjáratok foglalására szolgáló metódusokat.

Szállodafoglalási metódus:

- `@kernel_function(description="booking hotel")`: Egy dekorátor, amely a funkciót szállodafoglalásra szolgáló kernel funkcióként írja le.
- `def booking_hotel(self, query: Annotated[str, "A város neve"], check_in_date: Annotated[str, "Szállodai bejelentkezés ideje"], check_out_date: Annotated[str, "Szállodai kijelentkezés ideje"]) -> Annotated[str, "A szállodafoglalási információ eredményének visszaadása"]:`: Egy metódust definiál, amely szállodafoglalást végez, annotált paraméterekkel és visszatérési típussal.

A metódus egy paraméterekből álló szótárat hoz létre a szállodafoglalási kéréshez, majd GET kérést küld a SERP API-hoz. Ellenőrzi a válasz státuszát, és siker esetén visszaadja a szálloda tulajdonságait, sikertelenség esetén pedig None értéket ad vissza.

Repülőjárat-foglalási metódus:

- `@kernel_function(description="booking flight")`: Egy dekorátor, amely a funkciót repülőjárat-foglalásra szolgáló kernel funkcióként írja le.
- `def booking_flight(self, origin: Annotated[str, "Az indulási hely neve"], destination: Annotated[str, "Az érkezési hely neve"], outbound_date: Annotated[str, "Az indulás dátuma"], return_date: Annotated[str, "A visszatérés dátuma"]) -> Annotated[str, "A repülőjárat-foglalási információ eredményének visszaadása"]:`: Egy metódust definiál, amely repülőjáratokat foglal, annotált paraméterekkel és visszatérési típussal.

A metódus szótárakat hoz létre az induló és visszatérő járatok kéréséhez, majd GET kéréseket küld a SERP API-hoz. Ellenőrzi a válasz státuszát, és siker esetén hozzáfűzi a járatinformációt az eredmény szövegéhez, sikertelenség esetén pedig hibaüzenetet nyomtat. A metódus visszaadja az eredmény szöveget, amely tartalmazza a járatinformációkat.


In [ ]:
import requests

from typing import Annotated

from semantic_kernel.functions import kernel_function

# Define Booking Plugin
class BookingPlugin:
    """Booking Plugin for customers"""

    @kernel_function(description="booking hotel")
    def booking_hotel(
        self, 
        query: Annotated[str, "The name of the city"], 
        check_in_date: Annotated[str, "Hotel Check-in Time"], 
        check_out_date: Annotated[str, "Hotel Check-out Time"],
    ) -> Annotated[str, "Return the result of booking hotel information"]:
        """
        Function to book a hotel.
        Parameters:
        - query: The name of the city
        - check_in_date: Hotel Check-in Time
        - check_out_date: Hotel Check-out Time
        Returns:
        - The result of booking hotel information
        """

        # Define the parameters for the hotel booking request
        params = {
            "engine": "google_hotels",
            "q": query,
            "check_in_date": check_in_date,
            "check_out_date": check_out_date,
            "adults": "1",
            "currency": "GBP",
            "gl": "uk",
            "hl": "en",
            "api_key": SERP_API_KEY
        }

        # Send the GET request to the SERP API
        response = requests.get(BASE_URL, params=params)

        # Check if the request was successful
        if response.status_code == 200:
            # Parse the response content as JSON
            response = response.json()
            # Return the properties from the response
            return response["properties"]
        else:
            # Return None if the request failed
            return None

    @kernel_function(description="booking flight")
    def booking_flight(
        self, 
        origin: Annotated[str, "The name of Departure"], 
        destination: Annotated[str, "The name of Destination"], 
        outbound_date: Annotated[str, "The date of outbound"], 
        return_date: Annotated[str, "The date of Return_date"],
    ) -> Annotated[str, "Return the result of booking flight information"]:
        """
        Function to book a flight.
        Parameters:
        - origin: The name of Departure
        - destination: The name of Destination
        - outbound_date: The date of outbound
        - return_date: The date of Return_date
        - airline: The preferred airline carrier
        - hotel_brand: The preferred hotel brand
        Returns:
        - The result of booking flight information
        """
        
        # Define the parameters for the outbound flight request
        go_params = {
            "engine": "google_flights",
            "departure_id": "destination",
            "arrival_id": "origin",
            "outbound_date": "outbound_date",
            "return_date": "return_date",
            "currency": "GBP",
            "hl": "en",
            "airline": "airline",
            "hotel_brand": "hotel_brand",
            "api_key": "SERP_API_KEY"
        }
 
        print(go_params)

        # Send the GET request for the outbound flight
        go_response = requests.get(BASE_URL, params=go_params)

        # Initialize the result string
        result = ''

        # Check if the outbound flight request was successful
        if go_response.status_code == 200:
            # Parse the response content as JSON
            response = go_response.json()
            # Append the outbound flight information to the result
            result += "# outbound \n " + str(response)
        else:
            # Print an error message if the request failed
            print('error!!!')

        # Define the parameters for the return flight request
        back_params = {
            #"engine": "google_flights",
            "departure_id": destination,
            "arrival_id": origin,
            "outbound_date": outbound_date,
            "return_date": return_date,
            "currency": "GBP",
            "hl": "en",
            "api_key": SERP_API_KEY
        }

        # Send the GET request for the return flight
        back_response = requests.get(BASE_URL, params=back_params)

        # Check if the return flight request was successful
        if back_response.status_code == 200:
            # Parse the response content as JSON
            response = back_response.json()
            # Append the return flight information to the result
            result += "\n # return \n" + str(response)
        else:
            # Print an error message if the request failed
            print('error!!!')

        # Print the result
        print(result)

        # Return the result
        return result


# Magyarázat:
Importálási utasítások: Szükséges modulok importálása Azure hitelesítő adatokhoz, AI ügynökhöz, csevegési üzenet tartalmához, szerzői szerepkörhöz és kernel funkció dekorátorhoz.

Aszinkron kontextuskezelő: async with (DefaultAzureCredential() as creds, AzureAIAgent.create_client(credential=creds, conn_str="...") as client,): Ez egy aszinkron kontextuskezelőt állít be az Azure hitelesítő adatok kezelésére és egy AI ügynök kliens létrehozására.

Ügynök neve és utasításai:
- `AGENT_NAME = "BookingAgent"`: Meghatározza az ügynök nevét.
- `AGENT_INSTRUCTIONS = """..."""`: Részletes utasításokat ad az ügynöknek a foglalási kérések kezelésére.

Ügynökdefiníció létrehozása: `agent_definition = await client.agents.create_agent(...)`: Létrehoz egy ügynökdefiníciót a megadott modellel, névvel és utasításokkal.

AzureAI ügynök létrehozása: `agent = AzureAIAgent(...)`: Létrehoz egy AzureAI ügynököt a kliens, az ügynökdefiníció és a meghatározott plugin használatával.

Szál létrehozása: `thread: AzureAIAgentThread | None = None`: Szálat hoz létre az ügynök számára. Nem kötelező először szálat létrehozni - ha `None` értéket adunk meg, egy új szál jön létre az első meghívás során, és a válasz részeként visszaadódik.

Felhasználói bemenetek: `user_inputs = ["..."]`: Meghatározza a felhasználói bemenetek listáját, amelyeket az ügynöknek kell feldolgoznia.

A finally blokkban törölje a szálat és az ügynököt az erőforrások felszabadítása érdekében.


# Hitelesítés

A `DefaultAzureCredential` osztály az Azure SDK for Python része. Ez egy alapértelmezett módot biztosít az Azure szolgáltatásokkal való hitelesítéshez. Több módszerrel próbál meg hitelesíteni egy meghatározott sorrendben, például környezeti változókkal, kezelt identitással és Azure CLI hitelesítő adatokkal.

Aszinkron műveletek: Az aio modul jelzi, hogy a DefaultAzureCredential osztály támogatja az aszinkron műveleteket. Ez azt jelenti, hogy használhatod az asyncio-val nem blokkoló hitelesítési kérések végrehajtására.


In [ ]:
# Import necessary modules
from azure.identity.aio import DefaultAzureCredential
from semantic_kernel.agents import AzureAIAgent, AzureAIAgentSettings, AzureAIAgentThread

ai_agent_settings = AzureAIAgentSettings.create()

# Azure AI Setting
async with (
     DefaultAzureCredential() as creds,
    AzureAIAgent.create_client(
        credential=creds,
        conn_str=ai_agent_settings.project_connection_string.get_secret_value(),
    ) as client,
):    
    
    # Define the agent's name and instructions
    AGENT_NAME = "BookingAgent"
    AGENT_INSTRUCTIONS = """
    You are a booking agent, help me to book flights or hotels.

    Thought: Understand the user's intention and confirm whether to use the reservation system to complete the task.

    Action:
    - If booking a flight, convert the departure name and destination name into airport codes.
    - If booking a hotel or flight, use the corresponding API to call. Ensure that the necessary parameters are available. If any parameters are missing, use default values or assumptions to proceed.
    - If it is not a hotel or flight booking, respond with the final answer only.
    - Output the results using a markdown table:
    - For flight bookings, separate the outbound and return contents and list them in the order of Departure_airport Name | Airline | Flight Number | Departure Time | Arrival_airport Name | Arrival Time | Duration | Airplane | Travel Class | Price (USD) | Legroom | Extensions | Carbon Emissions (kg).
    - For hotel bookings, list them in the order of Properties Name | Properties description | check_in_time | check_out_time | prices | nearby_places | hotel_class | gps_coordinates.
    """

    # Create agent definition with the specified model, name, and instructions
    agent_definition = await client.agents.create_agent(
        model=ai_agent_settings.model_deployment_name,
        name=AGENT_NAME,
        instructions=AGENT_INSTRUCTIONS,
    )

    # Create the AzureAI Agent using the client and agent definition
    agent = AzureAIAgent(
        client=client,
        definition=agent_definition,
        plugins=[BookingPlugin()]
    )

    # Create a new thread for the agent
    # If no thread is provided, a new thread will be
    # created and returned with the initial response
    thread: AzureAIAgentThread | None = None

    # This is your prompt for the activity or task you want to complete 
    # Define user inputs for the agent to process we have provided some example prompts to test and validate 
    user_inputs = [
        # "Can you tell me the round-trip air ticket from  London to New York JFK aiport, the departure time is February 17, 2025, and the return time is February 23, 2025"
        # "Book a hotel in New York from Feb 20,2025 to Feb 24,2025"
        "Help me book flight tickets and hotel for the following trip London Heathrow LHR Feb 20th 2025 to New York JFK returning Feb 27th 2025 flying economy with British Airways only. I want a stay in a Hilton hotel in New York please provide costs for the flight and hotel"
        # "I have a business trip from London LHR to New York JFK on Feb 20th 2025 to Feb 27th 2025, can you help me to book a hotel and flight tickets"
    ]

    try:
        # Process each user input
        for user_input in user_inputs:
            print(f"# User: '{user_input}'")
            # Get the agent's response for the specified thread
            response = await agent.get_response(
                messages=user_input,
                thread=thread,
            )
            thread = response.thread
            # Print the agent's response
            print(f"{response.name}: '{response.content}'")
    finally:
        # Clean up by deleting the thread and agent
        await thread.delete() if thread else None
        await client.agents.delete_agent(agent.id)


---

**Felelősség kizárása**:  
Ez a dokumentum az AI fordítási szolgáltatás [Co-op Translator](https://github.com/Azure/co-op-translator) segítségével lett lefordítva. Bár törekszünk a pontosságra, kérjük, vegye figyelembe, hogy az automatikus fordítások hibákat vagy pontatlanságokat tartalmazhatnak. Az eredeti dokumentum az eredeti nyelvén tekintendő hiteles forrásnak. Kritikus információk esetén javasolt professzionális emberi fordítást igénybe venni. Nem vállalunk felelősséget semmilyen félreértésért vagy téves értelmezésért, amely a fordítás használatából eredhet.
